In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import cvxpy as cp
import numpy as np
from gcspy import GraphOfConvexSets

In [ ]:
gcs = GraphOfConvexSets()

v1 = gcs.add_vertex("v1")
x1 = v1.add_variable(2)
c1 = np.array([-3, 0])
v1.add_constraint(cp.norm(x1 - c1, np.inf) <= 1)
v1.add_cost(cp.sum_squares(x1 - c1))

v2 = gcs.add_vertex("v2")
x2 = v2.add_variable(2)
c2 = np.array([0, 2.5])
D2 = np.diag([.25, 2])
v2.add_constraint(cp.norm(D2 @ (x2 - c2), np.inf) <= 1)
v2.add_cost(cp.sum_squares(x2 - c2))

v3 = gcs.add_vertex("v3")
x3 = v3.add_variable(2)
c3 = np.array([3, 0])
v3.add_constraint(cp.norm(x3 - c3, 2) <= 1)
v3.add_cost(cp.sum_squares(x3 - c3))

v4 = gcs.add_vertex("v4")
x4 = v4.add_variable(2)
c4 = np.array([0, -2.5])
D4 = np.diag([1, 2])
v4.add_constraint(cp.norm(D4 @ (x4 - c4), 2) <= 1)
v4.add_cost(cp.sum_squares(x4 - c4))

v5 = gcs.add_vertex("v5")
x5 = v5.add_variable(2)
c5 = np.array([.3, .3])
v5.add_constraint(cp.norm(x5 - c5, 1) <= 1)
v5.add_cost(cp.sum_squares(x5 - c5))

In [ ]:
for tail in gcs.vertices:
    for head in gcs.vertices:
        if tail != head:
            edge = gcs.add_edge(tail, head)
            edge.add_cost(cp.norm(tail.variables[0] - head.variables[0], 2))

In [ ]:
gcs.graphviz()

In [ ]:
ilp_constraints = []
yv = gcs.vertex_binaries()
ye = gcs.edge_binaries()

for i, v in enumerate(gcs.vertices):
    inc_edges = gcs.incoming_indices(v)
    out_edges = gcs.outgoing_indices(v)

    ilp_constraints.append(yv[i] == 1)
    ilp_constraints.append(sum(ye[out_edges]) == 1)
    ilp_constraints.append(sum(ye[inc_edges]) == 1)

for r in range(2, gcs.num_vertices() - 1):
    for vertices in combinations(gcs.vertices, r):
        out_edges = gcs.outgoing_indices(vertices)
        ilp_constraints.append(sum(ye[out_edges]) >= 1)

In [ ]:
prob = gcs.solve_from_ilp(ilp_constraints)
print('Problem status:', prob.status)
print('Optimal value:', prob.value)
print('Vertex binaries:', {vertex.name: np.round(vertex.y.value, 4) for vertex in gcs.vertices})
print('Edge binaries:', {edge.name: np.round(edge.y.value, 4) for edge in gcs.edges})

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.gca().set_aspect('equal')
plt.axis('off')
gcs.plot_2d()
gcs.plot_subgraph_2d()
# plt.savefig('traveling_salesman.pdf')